# **Análise da Razão População/Empresas no Mercado Imobiliário Brasileiro**

## **Introdução**
Este notebook analisa a **razão entre a população de consumidores do mercado imobiliário** na faixa etária de **38 a 58 anos** e o **número de empresas ativas** no setor de construção em cada estado do Brasil. De acordo com a [SCOD Brasil](https://scod.com.br/blog/post/tend%C3%AAncias-do-mercado-imobili%C3%A1rio-para-2023), essa faixa etária representa aproximadamente **50% do mercado**.

A análise se baseia em dados históricos de **2007 a 2020** provenientes das seguintes fontes:
- **População na faixa etária de 38 a 58 anos**: Obtida da projeção populacional do IBGE [(link)](https://www.ibge.gov.br/estatisticas/sociais/populacao/9109-projecao-da-populacao.html).
- **Número de empresas ativas**: Extraído da **Tabela 1757** do IBGE/SIDRA, via requisição HTTP à API [(link)](https://apisidra.ibge.gov.br/home/ajuda).

A partir desses dados, aplicamos **técnicas de extrapolação** para prever os valores de **2021 e 2022**, permitindo a identificação de **tendências de mercado** e **potenciais oportunidades futuras**.

---

## **Objetivo**
Este estudo busca responder às seguintes questões:
1. **Quais estados apresentam um mercado saturado, estável ou com maior potencial de crescimento?**
2. **Qual a tendência da relação População/Empresas até 2022?**
3. **Como os estados podem ser agrupados com base nessa relação?**
4. **Quais estados possuem maior oportunidade de expansão para novas empresas do setor imobiliário?**

---

## **Etapas do Notebook**
### **1. Coleta de Dados**
- **População na faixa etária de 38 a 58 anos**: Obtida a partir da projeção populacional do IBGE.
- **Número de Empresas Ativas**: Obtido da **Tabela 1757 do SIDRA/IBGE** por meio de requisição HTTP direta.

### **2. Processamento dos Dados**
- Organização dos dados em uma **série temporal estruturada por estado (2007-2020)**.
- Análise exploratória, criação de gráficos e identificação de tendências.

### **3. Extrapolação para 2021 e 2022**
- Testamos **quatro métodos de extrapolação** para prever os valores futuros:
  - **Regressão Linear**
  - **Polinômio de Grau 4**
  - **Splines Cúbicos**
  - **Projeção Linear**
- Cada variável (**População e Empresas**) foi extrapolada separadamente, escolhendo o melhor método para cada uma com base no erro médio absoluto (MAE) e o erro percentual médio absoluto (MAPE).
- Após a extrapolação, foi realizado o **cálculo da Razão População/Empresas**.

### **4. Clusterização e Identificação de Oportunidades**
- **Agrupamento dos estados** em **três categorias** por meio do algoritmo **K-Means**:
  - **Saturado**: Alta razão População/Empresas.
  - **Estável**: Mercado equilibrado.
  - **Oportunidade**: Baixa razão População/Empresas, indicando potencial de crescimento.

### **5. Visualizações e Relatórios**
- **Gráficos interativos** para análise da evolução temporal e correlação entre variáveis.
- **Mapa do Brasil colorido por cluster**, destacando regiões de maior e menor oportunidade.
- **Exportação dos resultados em CSV**, permitindo análises futuras e reuso dos dados.

---

## **Resultados e relatório**




---
## Instalação de bibliotecas

In [ ]:
# Instalação de bibliotecas
!pip install --upgrade pip
!pip install -r requirements.txt

In [ ]:
import plotly.express as px
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error
from scipy.interpolate import CubicSpline
from sklearn.preprocessing import StandardScaler

---
## **1. Coleta de Dados**


## **Obtenção e Processamento da tabela da População (38 a 58 anos)**

### **A. Download dos Dados**
Os dados de população por idade e estado foram obtidos a partir da **Projeção da População 2024** do **IBGE** ([link](https://ftp.ibge.gov.br/Projecao_da_Populacao/Projecao_da_Populacao_2024/projecoes_2024_tab1_idade_simples.xlsx)) via requisição HTTP.

### **B. Processamento e Filtragem**
- **Anos considerados:** 2007 a 2022.
- **Faixa etária:** Apenas **38 a 58 anos**, pois representa cerca de **50% do mercado imobiliário**.
- **Sexo:** Apenas **"Ambos"**, pois já contém a soma de homens e mulheres.
- **Removido as agregações regionais e nacionais** para manter apenas estados individuais.
- **Agrupado os dados por estado e somamos a população**.

### **C. Salvar a tabela**
Foi gerado um arquivo contendo uma série temporal da população total na faixa etária alvo por estado.


In [ ]:
# A - Download dos Dados

# URL do arquivo - população por sexo e idade - 2000-2070
url = "https://ftp.ibge.gov.br/Projecao_da_Populacao/Projecao_da_Populacao_2024/projecoes_2024_tab1_idade_simples.xlsx"
file_name = "projecoes_2024_tab1_idade_simples.xlsx"

# Diretório onde o arquivo será salvo
data_dir = "data"
os.makedirs(data_dir, exist_ok=True)  # Garante que a pasta 'data' exista

# Caminho completo do arquivo
file_path = os.path.join(data_dir, file_name)

# Baixar o arquivo - população por sexo e idade - 2000-2070
response = requests.get(url)
if response.status_code == 200:
    with open(file_path, "wb") as file:
        file.write(response.content)
    print(f"Tabela salva em '{file_path}'")
else:
    print(f"Erro ao baixar a tabela. Status Code: {response.status_code}")

# Carregar os dados no pandas
try:
    df = pd.read_excel(file_path, sheet_name=0, skiprows=5)  # Carregando os dados em um dataframe, cortando o cabeçalho da tabela
    print("Dados carregados com sucesso!")
    print(df.head())  # Exibir as primeiras linhas do DataFrame
except FileNotFoundError:
    print(f"O arquivo '{file_path}' não foi encontrado.")
except Exception as e:
    print(f"Erro ao carregar o arquivo: {e}")


In [ ]:
# B - Processamento e Filtragem

# Define os anos de interesse (2007 a 2020)
years_of_interest = list(range(2007, 2021))

# Seleciona as colunas relevantes: 'IDADE', 'SEXO', 'SIGLA', 'LOCAL' e os anos de interesse
columns_of_interest = ['IDADE', 'SEXO', 'SIGLA', 'LOCAL'] + [year for year in years_of_interest]
filtered_df = df[columns_of_interest]

# Filtra as linhas com idades entre 38 e 58 anos
age_filtered_df = filtered_df[(filtered_df['IDADE'] >= 38) & (filtered_df['IDADE'] <= 58)]

# Filtra apenas as linhas onde o sexo é igual a 'Ambos'. Pois essas linhas já contêm a soma de 'Homens' e 'Mulheres'
both_sexes_df = age_filtered_df[age_filtered_df['SEXO'] == 'Ambos']

# Define a lista de siglas que devem ser removidas
unwanted_siglas = ['BR', 'NO', 'ND', 'SD', 'SU', 'CO']  # Federação e divisão por regiões

# Filtra as linhas cujo valor da coluna 'SIGLA' não está na lista de siglas indesejadas
filtered_siglas_df = both_sexes_df[~both_sexes_df['SIGLA'].isin(unwanted_siglas)]

# Agrupa por estado ('SIGLA') e soma a população para cada ano
state_population_totals = filtered_siglas_df.groupby('SIGLA').sum(numeric_only=True)

# Mostra os dados processados
state_population_totals.head(5)


In [ ]:
# B - Processamento e Filtragem

# Remove a coluna 'IDADE' após o agrupamento
state_population_totals_38to58 = state_population_totals.drop(columns=['IDADE'])
state_population_totals_38to58.head(5) # conferência e impressão do df final com 5 linhas

In [ ]:
#C - Salvar a tabela

# Feito o pré-processamento, agora salvar como um arquivo CSV
file_path = os.path.join(data_dir, "state_population_totals_38to58.csv")
state_population_totals_38to58.to_csv(file_path, index=True)


## **Obtenção e Processamento dos Dados de Empresas (SIDRA/IBGE)**

### **A. Acesso à API do SIDRA**
Os dados sobre o **número de empresas ativas na construção civil** foram obtidos a partir da **Tabela 1757 do SIDRA/IBGE**, acessada via **requisição HTTP**. A solicitação via API do SIDRA foi usada para consultar os dados entre **2007 e 2022**, selecionando **empresas com 5 ou mais funcionários**, pois os dados de **empresas menores são agrupados por regiões** e não estados (como desejado pela granularidade do problema).

### **B. Estrutura da Requisição**
A URL de consulta foi construída com os seguintes parâmetros:
- **Tabela:** `t/1757` (Empresas da Construção Civil)
- **Unidade Territorial:** `n3/all` (Todos os estados brasileiros)
- **Variável:** `v/410` (Número de empresas ativas)
- **Período:** `p/2007-2020` (Histórico de 2007 a 2020)
- **Classificação:** `c319/104030` (Apenas empresas com 5 ou mais funcionários)
- **Formato:** `f/n` (Removendo rótulos extras para facilitar o processamento)
- **Cabeçalho:** `h/n` (Removendo cabeçalhos desnecessários)

A URL final utilizada foi: https://apisidra.ibge.gov.br/values/t/1757/n3/all/v/410/p/2007-2022/c319/104030/f/n/h/n

### **C. Processamento dos Dados**
- A resposta da API foi convertida para um **DataFrame do Pandas**.
- **Selecionamos apenas as colunas relevantes:** Estado (`State`), Ano (`Year`) e Número de Empresas (`Number of Companies`).
- **Convertidos os valores numéricos** para garantir a consistência dos dados.

### **D. Salvar**
O resultado é uma série temporal estruturada do número de empresas ativas por estado entre 2007 e 2022.
- **Os dados processados foram salvos em um arquivo CSV** (`companies_5_or_more.csv`) para análise futura.

https://apisidra.ibge.gov.br/desctabapi.aspx?c=1757 --> Link com informações da tabela 1757



In [ ]:
# A - Acesso à API do SIDRA

# B - Estrutura da Requisição

# Define a URL para a API SIDRA (Tabela 1757, todos os estados, 2007-2020, número de empresas) 
url = "https://apisidra.ibge.gov.br/values/t/1757/n3/all/v/410/p/2007-2020/c319/104030/f/n/h/n"

# Envia a requisição GET para a API do SIDRA
response = requests.get(url)

# Verifica se a requisição foi bem-sucedida
if response.status_code == 200: # retorno que indica que a requisição foi bem sucedida
    print("Dados baixados com sucesso!\n\n")
    data = response.json()  # Converte a resposta para JSON
else:
    print(f"Erro ao baixar os dados: {response.status_code}")
    data = None

# C - Processamento dos Dados

# Processa os dados, se disponíveis
if data:
    # Converte os dados JSON para um DataFrame pandas
    df_companies = pd.DataFrame(data)

    # Exibe as primeiras linhas para entender a estrutura
    print("Dados brutos:")
    print(df_companies.head())

    # Filtra as colunas relevantes e renomeia para facilitar o uso
    df_cleaned = df_companies[['D1N', 'D3N', 'V']].copy()  
    df_cleaned.columns = ['State', 'Year', 'Number of Companies']
    
    # Converte Year e Number of Companies para o tipo correto usando .loc
    df_cleaned.loc[:, 'Year'] = pd.to_numeric(df_cleaned['Year'], errors='coerce')
    df_cleaned.loc[:, 'Number of Companies'] = pd.to_numeric(df_cleaned['Number of Companies'], errors='coerce')
    
    # Exibe os dados limpos
    print("\n\nDados limpos:")
    print(df_cleaned.head())

    # D - Salvar
    
    # Salva os dados em um arquivo CSV para análise futura
    file_path = os.path.join(data_dir, "companies_5_or_more.csv")
    df_cleaned.to_csv(file_path, index=False)
    print("Dados salvos no arquivo 'companies_5_or_more.csv'.")



---
## **Etapa 2: Processamento dos Dados**

Nesta etapa, os dados de **população** e **número de empresas ativas** são organizados, processados e analisados para preparar a extrapolação e identificar padrões. O objetivo é calcular a **razão população/empresas** e entender a **distribuição** entre os estados.

### **A. Cálculo da Razão População/Empresas**
- Os dados de **população na faixa de 38 a 58 anos** e **empresas do setor de construção** são carregados e combinados.
- Para cada estado e ano, foi calculada a **razão entre a população e o número de empresas ativas**.

### **B. Análise Exploratória e Visualizações**
- **Gráfico de Linha**: Mostra a evolução da razão **População/Empresas** ao longo dos anos para cada estado, ajudando a identificar tendências temporais.
- **Gráfico de Barras**: Mostra a **razão média por estado** entre 2007 e 2020, permitindo comparações diretas.
- **Gráfico de Dispersão**: Mostra a **correlação entre população e número de empresas**, ajudando a identificar padrões e estados com maior ou menor potencial.

### **C. Cálculo da Correlação**
- A correlação entre **população e número de empresas** foi calculada para cada estado.
- **Correlação Positiva**: Indica que o número de empresas cresce conforme a população na faixa etária analisada aumenta.
- **Correlação Fraca**: Indica que o crescimento da população e o número de empresas não têm relação direta.
- **Correlação Negativa**: Indica que o número de empresas pode estar diminuindo enquanto a população da faixa etária analisada aumenta.

### D. Clusterização (2008 - 2020)


In [ ]:
# A - Cálculo da Razão População/Empresas

# Carregar os arquivos CSV 
file_path_companies = os.path.join(data_dir, "companies_5_or_more.csv")
file_path_population = os.path.join(data_dir, "state_population_totals_38to58.csv")
df_companies = pd.read_csv(file_path_companies)  # Empresas com 5 ou mais pessoas (2007 a 2020)
df_population = pd.read_csv(file_path_population)  # População de 38 a 58 anos (2007 a 2020)


# Verificação dos dados
print("Dados de empresas:")
print(df_companies.head(3))

print("\nDados de população:")
print(df_population.head(3))



In [ ]:

# dicionário para mapear nomes completos para siglas
state_to_sigla = {
    "Acre": "AC", "Alagoas": "AL", "Amapá": "AP", "Amazonas": "AM", "Bahia": "BA",
    "Ceará": "CE", "Distrito Federal": "DF", "Espírito Santo": "ES", "Goiás": "GO",
    "Maranhão": "MA", "Mato Grosso": "MT", "Mato Grosso do Sul": "MS", "Minas Gerais": "MG",
    "Pará": "PA", "Paraíba": "PB", "Paraná": "PR", "Pernambuco": "PE", "Piauí": "PI",
    "Rio de Janeiro": "RJ", "Rio Grande do Norte": "RN", "Rio Grande do Sul": "RS",
    "Rondônia": "RO", "Roraima": "RR", "Santa Catarina": "SC", "São Paulo": "SP",
    "Sergipe": "SE", "Tocantins": "TO"
}

# Converte os nomes dos estados em df_companies para siglas
df_companies['SIGLA'] = df_companies['State'].map(state_to_sigla)

# Transforma df_population para um formato longo (anos em uma única coluna)
df_population_long = df_population.melt(id_vars=['SIGLA'], 
                                        var_name='Year', 
                                        value_name='Population')

# Converte a coluna 'Year' para inteiro
df_population_long['Year'] = pd.to_numeric(df_population_long['Year'])

# Combina os dois datasets usando SIGLA e Year como chave
combined_df = pd.merge(df_population_long, df_companies, on=['SIGLA', 'Year'], how='inner')

# Calcula a razão população/empresas
combined_df['Population-to-Companies Ratio'] = combined_df['Population'] / combined_df['Number of Companies']

# Exibe os dados combinados
print("Dados combinados:")
print(combined_df.head(5))

# Salva os dados em um novo arquivo CSV
file_path = os.path.join(data_dir, "combined_population_to_companies.csv")
combined_df.to_csv(file_path, index=False)
print("Dados salvos no arquivo 'combined_population_to_companies.csv'.")


### **B. Análise Exploratória e Visualizações**

- **Gráfico de Linha**: Mostra a evolução da razão **População/Empresas** ao longo dos anos para cada estado, ajudando a identificar tendências temporais.
- **Gráfico de Barras**: Mostra a **razão média por estado** entre 2007 e 2020, permitindo comparações diretas.
- **Gráfico de Dispersão**: Mostra a **correlação entre população e número de empresas**, ajudando a identificar padrões e estados com maior ou menor potencial.


In [ ]:

# B - Análise Exploratória e Visualizações

# Carregue o dataset combinado
file_path = os.path.join(data_dir, "combined_population_to_companies.csv")
combined_df = pd.read_csv(file_path)

# 1. Gráfico Interativo de Linha: Evolução da Razão por Estado ao Longo dos Anos
fig_line = px.line(
    combined_df,
    x="Year",
    y="Population-to-Companies Ratio",
    color="SIGLA",  # Uma cor diferente para cada estado
    title="Evolução da Razão População/Empresas por Estado (2007-2020)",
    labels={"Population-to-Companies Ratio": "Razão População/Empresas"},
    hover_name="SIGLA"  # Exibe a sigla do estado ao passar o mouse
)
fig_line.update_layout(hovermode="x unified")
fig_line.show()

# 2. Gráfico Interativo de Barras: Razão Média por Estado
mean_ratio_by_state = combined_df.groupby('SIGLA')['Population-to-Companies Ratio'].mean().reset_index()

fig_bar = px.bar(
    mean_ratio_by_state,
    x="SIGLA",
    y="Population-to-Companies Ratio",
    title="Razão Média População/Empresas por Estado (2007-2020)",
    labels={"Population-to-Companies Ratio": "Razão Média", "SIGLA": "Estado"},
    color="SIGLA",  # Uma cor diferente para cada estado
    hover_name="SIGLA"  # Exibe a sigla ao passar o mouse
)
fig_bar.update_layout(showlegend=False)  # Remove a legenda, já que as cores indicam os estados
fig_bar.show()

# 3. Gráfico Interativo de Dispersão: Correlação entre População e Número de Empresas
fig_scatter = px.scatter(
    combined_df,
    x="Number of Companies",
    y="Population",
    color="SIGLA",  # Cor por estado
    title="Correlação entre População e Número de Empresas",
    labels={"Number of Companies": "Número de Empresas", "Population": "População"},
    hover_name="SIGLA"  # Exibe a sigla ao passar o mouse
)
fig_scatter.show()



### **C. Cálculo da Correlação**
- **Correlação Positiva (> 0.20)**: Indica que o número de empresas cresce conforme a população na faixa etária analisada aumenta (azul).
- **Correlação Fraca (-0.20 a 0.20)**: Indica que o crescimento da população e o número de empresas não têm relação direta (verde).
- **Correlação Negativa (< -0.20)**: Indica que o número de empresas pode estar diminuindo enquanto a população da faixa etária analisada aumenta (vermelha).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Lista para armazenar os resultados
correlation_results = []

# Itera por cada grupo (estado) e calcula a correlação
for state, group in combined_df.groupby('SIGLA'):
    correlation = group['Population'].corr(group['Number of Companies'])
    correlation_results.append({'State': state, 'Correlation': correlation})

# Converte os resultados para um DataFrame
correlation_by_state = pd.DataFrame(correlation_results)

# Exibe a tabela com a correlação por estado
print("Correlação entre População e Número de Empresas por Estado:")
print(correlation_by_state)

# Salva a tabela em CSV, se necessário
file_path = os.path.join(data_dir, "correlation_by_state.csv")
correlation_by_state.to_csv(file_path, index=False)

# Define uma função para atribuir cores com base nos valores de correlação
def assign_color(value):
    if value > 0.20:
        return 'blue'  # Correlação forte positiva
    elif value < -0.20:
        return 'red'  # Correlação forte negativa
    else:
        return 'green'  # Correlação neutra

# Cria uma coluna de cores no DataFrame
correlation_by_state['Color'] = correlation_by_state['Correlation'].apply(assign_color)

# Criar um mapeamento para os estados com base nas cores atribuídas
plt.figure(figsize=(8, 5))
sns.barplot(
    data=correlation_by_state,
    x='State',
    y='Correlation',
    hue='State',  # Define o mapeamento de cores com base na sigla do estado
    palette={state: color for state, color in zip(correlation_by_state['State'], correlation_by_state['Color'])},
    dodge=False
)

plt.title('Correlação entre População e Número de Empresas por Estado')
plt.xlabel('Estado')
plt.ylabel('Correlação')
plt.xticks(rotation=45)
plt.axhline(0, color='black', linewidth=0.8, linestyle='--')  # Linha indicando correlação neutra
plt.legend([], [], frameon=False)  # Remove a legenda
plt.tight_layout()
plt.show()





### **D. Clusterização (2008 - 2020)**

Nesta etapa, foi realizada a clusterização utilizando o K-Means (K=3) para identificar padrões de mercado.

---

## **O que foi feito**
1. **Os dados foram transformados** em uma série temporal (2007-2020) contendo a razão População/Empresas para cada estado.
2. **O algoritmo K-Means foi aplicado** para agrupar o estado em três categorias:
   - **Saturado**: Alta razão População/Empresas, indicando mercado consolidado.
   - **Estável**: Mercado equilibrado.
   - **Oportunidade**: Baixa razão, sugerindo potencial de crescimento.
3. **Os clusters foram analisados**, identificando a qual grupo o estado pertence.
4. **As visualizações foram geradas**, incluindo:
   - Um **gráfico de dispersão** para visualizar a evolução temporal dos clusters.
   - Um **gráfico de correlação entre População e Empresas**.
   - Um **resumo estatístico** da média de população e empresas por cluster.


In [ ]:
# Gráfico de dispersão

# Carregar o dataset combinado
file_path = os.path.join(data_dir, "combined_population_to_companies.csv")
combined_df = pd.read_csv(file_path)

# Filtrar os dados para o período de 2007 a 2020
filtered_df = combined_df[(combined_df['Year'] >= 2007) & (combined_df['Year'] <= 2020)]

# Pivotar os dados para que cada estado seja uma linha e cada ano uma coluna
pivoted_df = filtered_df.pivot(index='SIGLA', columns='Year', values='Population-to-Companies Ratio')

# Preencher valores ausentes
pivoted_df.fillna(0, inplace=True)

# Aplicar K-Means para clusterizar os estados com base nas séries temporais
n_clusters = 3  # Número de clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
pivoted_df['Cluster'] = kmeans.fit_predict(pivoted_df)

# Adicionar os clusters ao dataframe original
filtered_df = filtered_df.merge(pivoted_df['Cluster'], on='SIGLA')

# Identificar estados saturados e com oportunidades
saturated_states = pivoted_df[pivoted_df['Cluster'] == pivoted_df['Cluster'].max()].index.tolist()
opportunity_states = pivoted_df[pivoted_df['Cluster'] == pivoted_df['Cluster'].min()].index.tolist()

# Visualizar os clusters em um gráfico de dispersão
plt.figure(figsize=(8, 5))
sns.scatterplot(
    data=filtered_df,
    x='Year',
    y='Population-to-Companies Ratio',
    hue='Cluster',
    palette='tab10'
)
plt.title('Clusterização dos Estados com Base na Razão População/Empresas')
plt.xlabel('Ano')
plt.ylabel('Razão População/Empresas')
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()

# Resultados
print("Estados Saturados:", saturated_states)
print("Estados com Oportunidades:", opportunity_states)

# Salvar os resultados para análise posterior
file_path = os.path.join(data_dir, "clustered_states.csv")
pivoted_df.to_csv(file_path)



In [ ]:
# Gráfico de correlação

# Carregar o dataset combinado
file_path = os.path.join(data_dir, "combined_population_to_companies.csv")
combined_df = pd.read_csv(file_path)

# Filtrar os dados para o período de 2007 a 2020
filtered_df = combined_df[(combined_df['Year'] >= 2007) & (combined_df['Year'] <= 2020)]

# Preparar os dados para clusterização
pivoted_df = filtered_df.pivot(index='SIGLA', columns='Year', values='Population-to-Companies Ratio')

# Aplicar K-Means
n_clusters = 3  # Número de clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
pivoted_df['Cluster'] = kmeans.fit_predict(pivoted_df)

# Adicionar os clusters ao DataFrame original
filtered_df = filtered_df.merge(pivoted_df['Cluster'], on='SIGLA')

# Gráfico de Dispersão: Empresas x População por Cluster
plt.figure(figsize=(8, 5))
sns.scatterplot(
    data=filtered_df,
    x='Population',
    y='Number of Companies',
    hue='Cluster',
    palette='tab10',
    style='Cluster',  # Diferenciar clusters também por estilo
    s=100  # Tamanho dos pontos
)
plt.title('Clusters de Estados: Número de Empresas vs População')
plt.xlabel('População')
plt.ylabel('Número de Empresas')
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()

# Tabela Resumida por Cluster
cluster_summary = filtered_df.groupby('Cluster').agg({
    'Population': 'mean',  # Média da população
    'Number of Companies': 'mean',  # Média do número de empresas
    'SIGLA': 'nunique'  # Contar estados únicos
}).rename(columns={'SIGLA': 'Number of States'})

# resumo
print("\nResumo por Cluster (Corrigido):")
print(cluster_summary)

# Salvar os clusters e a tabela para análise posterior
file_path = os.path.join(data_dir, "state_clusters.csv")
filtered_df.to_csv(file_path, index=False)
file_path = os.path.join(data_dir, "cluster_summary_corrected.csv")
cluster_summary.to_csv(file_path, index=True)




In [ ]:
# Estados em cada cluster
states_in_clusters = filtered_df[['SIGLA', 'Cluster']].drop_duplicates().sort_values('Cluster')
print(states_in_clusters)


# **Etapa 3: Extrapolação para 2021 e 2022**

Nesta etapa, foi realizada a **previsão dos valores para 2021 e 2022**, utilizando diferentes abordagens de extrapolação para **população** e **número de empresas** separadamente.

---

## **O que foi feito**
1. **Definição dos Métodos de Extrapolação**:
   - **Regressão Linear**
   - **Polinômio de Grau 4**
   - **Splines Cúbicos**
   - **Projeção Linear**
   
2. **Avaliação do Melhor Método**:
   - Para cada estado, os métodos foram testados individualmente para população e empresas.
   - O erro médio absoluto (MAE) e o erro percentual médio absoluto (MAPE) foram utilizados para escolher o método mais preciso.

3. **Extrapolação Separada para População e Empresas**:
   - Os valores de população foram extrapolados considerando tendências individuais de cada estado.
   - O número de empresas ativas também foi extrapolado separadamente, garantindo previsões mais coerentes.

4. **Cálculo da Razão População/Empresas**:
   - Após a extrapolação, a razão População/Empresas foi recalculada com os valores previstos.

5. **Duas Abordagens para Extrapolação**:
   - **Histórico Completo (2007-2020)**: Captura tendências de longo prazo.
   - **Janela de 5 anos (2016-2020) e (2017-2021)**: Reflete a tendência atual, reduzindo a influência de eventos passados.


In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# 1. Carregar os dados
file_path = os.path.join(data_dir, "combined_population_to_companies.csv")
df = pd.read_csv(file_path)

# 2. Filtrar os dados entre 2007 e 2020
df_filtered = df[df['Year'].between(2007, 2020)]

# 3. Separar população e número de empresas
df_population = df_filtered.pivot(index='SIGLA', columns='Year', values='Population')
df_companies = df_filtered.pivot(index='SIGLA', columns='Year', values='Number of Companies')

# Definir graus para testar
degrees = [1, 2, 3, 4]

# Dicionários para armazenar os erros para cada estado e grau
error_results_population = {}
error_results_companies = {}

# Função para calcular RMSE e escolher o melhor grau
def evaluate_extrapolation(df, error_results):
    for state in df.index:
        years = np.array(df.columns[df.columns <= 2020])  # Apenas anos conhecidos
        values = df.loc[state, years].values

        if np.count_nonzero(~np.isnan(values)) >= 3:  # Mínimo 3 pontos para ajuste
            error_results[state] = {}
            for deg in degrees:
                coef = np.polyfit(years, values, deg=deg)
                poly = np.poly1d(coef)
                predicted = poly(years)  # Previsão para os anos conhecidos

                mse = mean_squared_error(values, predicted)  # Calcula o erro médio quadrático
                rmse = np.sqrt(mse)  # Calcula o erro médio quadrático raiz (RMSE)
                error_results[state][f'RMSE - Grau {deg}'] = rmse

    # Criar DataFrame para visualizar os erros
    df_errors = pd.DataFrame(error_results).T

    # Determinar o melhor grau para cada estado com base no menor RMSE
    df_errors['Melhor Grau'] = df_errors.idxmin(axis=1).str.extract(r'(\d+)').astype(int)

    return df_errors

# Avaliar extrapolação para população e empresas
df_errors_population = evaluate_extrapolation(df_population, error_results_population)
df_errors_companies = evaluate_extrapolation(df_companies, error_results_companies)

# Exibir os resultados
print("\nMatriz de RMSE por Grau para Cada Estado (População):")
print(df_errors_population)
print("\nMatriz de RMSE por Grau para Cada Estado (Empresas):")
print(df_errors_companies)

# Salvar os resultados em CSV para análise posterior
file_path = os.path.join(data_dir, "prediction_rmse_by_degree_population.csv")
df_errors_population.to_csv(file_path, index=True)
file_path = os.path.join(data_dir, "prediction_rmse_by_degree_companies.csv")
df_errors_companies.to_csv(file_path, index=True)



#### Será usado um polinômio de grau 4 para extrapolar os pontos. 

In [ ]:

# Extrapolação com janela de 5 anos

# 1. Função de Extrapolação e Avaliação
def extrapolate_series(df_pivot, method, years_to_predict=[2021, 2022], window_size=5):
    """
    Extrapola séries temporais usando diferentes métodos e avalia com MAE e MAPE.
    Utiliza um histórico de 'window_size' anos para prever os próximos anos.
    
    Parâmetros:
    - df_pivot (pd.DataFrame): DataFrame pivotado com índices como estados e colunas como anos.
    - method (str): Método de extrapolação ('linear_regression', 'polynomial', 'spline', 'linear_projection').
    - years_to_predict (list): Lista de anos para prever.
    - window_size (int): Número de anos históricos usados para prever o próximo ano.
    
    Retorna:
    - pd.DataFrame: DataFrame com extrapolações.
    - dict: Dicionário com métricas MAE e MAPE.
    """
    # Cria uma cópia do DataFrame para armazenar os resultados extrapolados
    df_result = df_pivot.copy()
    
    # Inicializa o dicionário para armazenar as métricas de avaliação
    metrics = {'MAE': [], 'MAPE': []}
    
    # Itera sobre cada estado no DataFrame pivotado
    for state in df_pivot.index:
        # Obtém os valores da série temporal para o estado atual
        values = df_pivot.loc[state].values
        years = np.array(df_pivot.columns)
        
        # Filtra os valores válidos (não NaN e finitos)
        valid_indices = ~np.isnan(values) & np.isfinite(values)
        years_valid = years[valid_indices]
        values_valid = values[valid_indices]
        
        # Verifica se há dados suficientes para extrapolação
        if len(years_valid) >= window_size:
            # Inicializa uma lista para armazenar os valores extrapolados
            extrapolated_values = []
            
            # Itera sobre cada ano a ser previsto
            for i, year in enumerate(years_to_predict):
                # Define a janela de treino
                if i == 0:
                    train_years = years_valid[-window_size:]
                    train_values = values_valid[-window_size:]
                else:
                    # Inclui os valores extrapolados anteriores
                    train_years = np.append(train_years[1:], years_to_predict[i-1])
                    train_values = np.append(train_values[1:], extrapolated_values[-1])
                
                try:
                    # Aplica o método de extrapolação selecionado
                    if method == "linear_regression":
                        # Regressão Linear
                        coef = np.polyfit(train_years, train_values, deg=1)
                        poly = np.poly1d(coef)
                        extrapolated = poly(year)
                    
                    elif method == "polynomial":
                        # Regressão Polinomial de Grau 4
                        coef = np.polyfit(train_years, train_values, deg=4)
                        poly = np.poly1d(coef)
                        extrapolated = poly(year)
                    
                    elif method == "spline":
                        # Spline Cúbico
                        cs = CubicSpline(train_years, train_values, extrapolate=True)
                        extrapolated = cs(year)
                    
                    elif method == "linear_projection":
                        # Projeção Linear baseada na taxa média anual de crescimento
                        growth_rate = (train_values[-1] - train_values[0]) / (train_years[-1] - train_years[0])
                        extrapolated = train_values[-1] + growth_rate * (year - train_years[-1])
                    
                    # Adiciona o valor extrapolado à lista
                    extrapolated_values.append(extrapolated)
                    
                    # Atualiza o DataFrame de resultados com o valor extrapolado
                    df_result.loc[state, year] = extrapolated
                
                except Exception as e:
                    # Em caso de erro, insere NaN no ano extrapolado e armazena NaN nas métricas
                    print(f"Erro ao extrapolar para o estado {state} no ano {year} com método {method}: {e}")
                    df_result.loc[state, year] = np.nan
                    extrapolated_values.append(np.nan)
                    metrics['MAE'].append(np.nan)
                    metrics['MAPE'].append(np.nan)
                    continue  # Pula para o próximo ano
                
                # Calcula as métricas apenas para o primeiro ano extrapolado (2021)
                if i == 0:
                    if not np.isnan(extrapolated):
                        mae = mean_absolute_error([values_valid[-1]], [extrapolated])
                        mape = np.mean(np.abs((values_valid[-1] - extrapolated) / values_valid[-1])) * 100
                        metrics['MAE'].append(mae)
                        metrics['MAPE'].append(mape)
                    else:
                        metrics['MAE'].append(np.nan)
                        metrics['MAPE'].append(np.nan)
            
            # Para 2022, caso tenha extrapolado 2021 com sucesso
            if len(years_to_predict) > 1:
                year = years_to_predict[1]
                if not np.isnan(extrapolated_values[-1]):
                    mae = mean_absolute_error([values_valid[-2]], [extrapolated_values[-1]])
                    mape = np.mean(np.abs((values_valid[-2] - extrapolated_values[-1]) / values_valid[-2])) * 100
                    metrics['MAE'].append(mae)
                    metrics['MAPE'].append(mape)
                else:
                    metrics['MAE'].append(np.nan)
                    metrics['MAPE'].append(np.nan)
        
        else:
            # Se houver poucos dados, insere NaN nos anos extrapolados e armazena NaN nas métricas
            for year in years_to_predict:
                df_result.loc[state, year] = np.nan
                metrics['MAE'].append(np.nan)
                metrics['MAPE'].append(np.nan)
    
    # Calcula as métricas médias, ignorando NaNs
    avg_metrics = {
        'MAE': np.nanmean(metrics['MAE']),
        'MAPE': np.nanmean(metrics['MAPE'])
    }
    
    return df_result, avg_metrics

# 2. Carregamento e Preparação dos Dados
input_file = "combined_population_to_companies.csv"

# Carrega os dados do CSV
file_path = os.path.join(data_dir, input_file)
df = pd.read_csv(file_path)

# Filtra os dados para os anos entre 2007 e 2020
df_filtered = df[df['Year'].between(2007, 2020)].copy()

# Verifica se todas as colunas necessárias estão presentes no DataFrame
required_columns = ['SIGLA', 'Year', 'Population', 'Number of Companies']
for col in required_columns:
    if col not in df_filtered.columns:
        raise ValueError(f"A coluna '{col}' está faltando no arquivo CSV.")

# Pivotar os dados separadamente para População e Número de Empresas
df_population = df_filtered.pivot(index='SIGLA', columns='Year', values='Population').astype(float)
df_companies = df_filtered.pivot(index='SIGLA', columns='Year', values='Number of Companies').astype(float)

# Garante que os índices das colunas (anos) sejam inteiros
df_population.columns = df_population.columns.astype(int)
df_companies.columns = df_companies.columns.astype(int)

# 3. Definição dos Métodos de Extrapolação a Serem Utilizados
methods = ['linear_regression', 'polynomial', 'spline', 'linear_projection']

# 4. Inicialização de Dicionários para Armazenar Métricas e Resultados Extrapolados
mae_population = {}
mape_population = {}
extrapolated_population = {}

mae_companies = {}
mape_companies = {}
extrapolated_companies = {}

# 5. Aplicar Extrapolação para População Usando Cada Método
for method in methods:
    # Extrapola a População usando o método atual
    extrapolated_population[method], metrics = extrapolate_series(df_population, method)
    # Armazena as métricas MAE e MAPE
    mae_population[method] = metrics['MAE']
    mape_population[method] = metrics['MAPE']

# 6. Aplicar Extrapolação para Número de Empresas Usando Cada Método
for method in methods:
    # Extrapola o Número de Empresas usando o método atual
    extrapolated_companies[method], metrics = extrapolate_series(df_companies, method)
    # Armazena as métricas MAE e MAPE
    mae_companies[method] = metrics['MAE']
    mape_companies[method] = metrics['MAPE']

# 7. Seleção do Melhor Método com Base nas Métricas (MAE e MAPE)
def select_best_method(metrics_df):
    """
    Seleciona o melhor método com base em múltiplas métricas (MAE e MAPE).
    Escolhe o método com menor MAE e, em caso de empate, menor MAPE.
    
    Parâmetros:
    - metrics_df (pd.DataFrame): DataFrame com as métricas para cada método.
    
    Retorna:
    - str: Nome do melhor método.
    - pd.DataFrame: DataFrame com as métricas ordenadas.
    """
    # Ordena o DataFrame primeiro por MAE e depois por MAPE
    metrics_sorted = metrics_df.sort_values(by=['MAE', 'MAPE'])
    
    # Seleciona o método com a menor MAE e, em caso de empate, a menor MAPE
    best_method = metrics_sorted.iloc[0]['Method']
    
    return best_method, metrics_sorted

# Combina as métricas em DataFrames para População
metrics_population_df = pd.DataFrame({
    'Method': list(methods),
    'MAE': [mae_population[m] for m in methods],
    'MAPE': [mape_population[m] for m in methods]
})

# Combina as métricas em DataFrames para Empresas
metrics_companies_df = pd.DataFrame({
    'Method': list(methods),
    'MAE': [mae_companies[m] for m in methods],
    'MAPE': [mape_companies[m] for m in methods]
})

# Seleciona o melhor método para População
best_population_method, metrics_population_df = select_best_method(metrics_population_df)

# Seleciona o melhor método para Número de Empresas
best_companies_method, metrics_companies_df = select_best_method(metrics_companies_df)

# 8. Imprime as Métricas e o Melhor Método Selecionado
print("\n### MAE e MAPE Médio por Método (População): ###")
print(metrics_population_df[['Method', 'MAE', 'MAPE']])

print(f"\nMelhor Método para População: {best_population_method.replace('_', ' ').capitalize()}")

print("\n### MAE e MAPE Médio por Método (Número de Empresas): ###")
print(metrics_companies_df[['Method', 'MAE', 'MAPE']])

print(f"\nMelhor Método para Número de Empresas: {best_companies_method.replace('_', ' ').capitalize()}")

# 9. Seleciona os Resultados Extrapolados Usando os Melhores Métodos
best_population = extrapolated_population[best_population_method].reset_index()
best_companies = extrapolated_companies[best_companies_method].reset_index()

# 10. Transforma os Dados para o Formato Longo (Tidy) para Facilitar a Combinação
best_population_melted = best_population.melt(id_vars=["SIGLA"], var_name="Year", value_name="Population")
best_companies_melted = best_companies.melt(id_vars=["SIGLA"], var_name="Year", value_name="Number of Companies")

# 11. Combina os Dados Extrapolados de População e Número de Empresas
df_final = pd.merge(best_population_melted, best_companies_melted, on=["SIGLA", "Year"], how="left")

# 12. Calcula a Razão População/Empresas
df_final["Population-to-Companies Ratio"] = df_final["Population"] / df_final["Number of Companies"]

# 13. Correção de Valores Infinitos e Tipos de Dados
# Identifica as colunas numéricas para aplicar as correções
numeric_cols = df_final.select_dtypes(include=[np.number]).columns.tolist()

# Substitui valores infinitos por NaN apenas nas colunas numéricas
df_final[numeric_cols] = df_final[numeric_cols].replace([np.inf, -np.inf], np.nan)

# Garante que as colunas numéricas são do tipo float antes da interpolação
df_final[numeric_cols] = df_final[numeric_cols].astype(float)

# Interpola dados faltantes utilizando interpolação linear apenas nas colunas numéricas
df_final[numeric_cols] = df_final[numeric_cols].interpolate(method='linear', inplace=False)

# 14. Salva os Resultados Extrapolados em um Arquivo CSV
output_file = "extrapolated_population_companies_separate_methods.csv"
file_path = os.path.join(data_dir, output_file)
df_final.to_csv(file_path, index=False)
print(f"\nExtrapolação salva em '{output_file}'")

# 15. Geração de Gráficos da Razão População/Empresas ao Longo dos Anos
output_dir = "charts_ratio"
os.makedirs(output_dir, exist_ok=True)  # Cria a pasta se não existir

# Itera sobre cada estado para gerar e salvar os gráficos
for state in df_final["SIGLA"].unique():
    df_state = df_final[df_final["SIGLA"] == state].sort_values("Year")
    
    # Separa os anos históricos e extrapolados
    historical = df_state["Year"] <= 2020
    extrapolated = df_state["Year"] > 2020
    
    plt.figure(figsize=(10, 5))
    
    # Plotagem dos anos históricos
    plt.plot(df_state.loc[historical, "Year"], df_state.loc[historical, "Population-to-Companies Ratio"],
             'o-b', label="Histórico")
    
    # Plotagem dos anos extrapolados com linhas tracejadas vermelhas
    plt.plot(df_state.loc[extrapolated, "Year"], df_state.loc[extrapolated, "Population-to-Companies Ratio"],
             's--r', label="Extrapolado")
    
    # Conexão entre 2020 e 2021 com linha tracejada vermelha
    if extrapolated.any():
        last_historical_year = df_state.loc[historical, "Year"].max()
        first_extrapolated_year = df_state.loc[extrapolated, "Year"].min()
        if last_historical_year < first_extrapolated_year:
            last_historical_value = df_state.loc[df_state["Year"] == last_historical_year, "Population-to-Companies Ratio"].values[0]
            first_extrapolated_value = df_state.loc[df_state["Year"] == first_extrapolated_year, "Population-to-Companies Ratio"].values[0]
            plt.plot([last_historical_year, first_extrapolated_year],
                     [last_historical_value, first_extrapolated_value],
                     'r--')
    
    plt.title(f"Evolução da Razão População/Empresas - {state}")
    plt.xlabel("Ano")
    plt.ylabel("Razão População/Empresas")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    
    # Salva o gráfico na pasta especificada
    plt.savefig(f"{output_dir}/{state}_ratio.png")
    plt.close()

print(f"Gráficos da Razão População/Empresas salvos na pasta '{output_dir}'")



In [ ]:
#Extrapolação com o histórico total

# 1. Função de Extrapolação e Avaliação
def extrapolate_series(df_pivot, method, years_to_predict=[2021, 2022]):
    """
    Extrapola séries temporais usando diferentes métodos e avalia com MAE e MAPE.
    
    Parâmetros:
    - df_pivot (pd.DataFrame): DataFrame pivotado com índices como estados e colunas como anos.
    - method (str): Método de extrapolação ('linear_regression', 'polynomial', 'spline', 'linear_projection').
    - years_to_predict (list): Lista de anos para prever.
    
    Retorna:
    - pd.DataFrame: DataFrame com extrapolações.
    - dict: Dicionário com métricas MAE e MAPE.
    """
    # Cria uma cópia do DataFrame para armazenar os resultados extrapolados
    df_result = df_pivot.copy()
    
    # Inicializa o dicionário para armazenar as métricas de avaliação
    metrics = {'MAE': [], 'MAPE': []}
    
    # Itera sobre cada estado no DataFrame pivotado
    for state in df_pivot.index:
        # Obtém os valores da série temporal para o estado atual
        values = df_pivot.loc[state].values
        years = np.array(df_pivot.columns)
        
        # Filtra os valores válidos (não NaN e finitos)
        valid_indices = ~np.isnan(values) & np.isfinite(values)
        years_valid = years[valid_indices]
        values_valid = values[valid_indices]
        
        # Verifica se há dados suficientes para extrapolação
        if len(years_valid) >= 4:
            # Divide os dados em treino e teste (deixa os dois últimos anos para teste)
            train_years = years_valid[:-2]
            train_values = values_valid[:-2]
            test_years = years_valid[-2:]
            test_values = values_valid[-2:]
            
            try:
                # Aplica o método de extrapolação selecionado
                if method == "linear_regression":
                    # Regressão Linear
                    coef = np.polyfit(train_years, train_values, deg=1)
                    poly = np.poly1d(coef)
                    pred = poly(test_years)
                    extrapolated = poly(years_to_predict)
                
                elif method == "polynomial":
                    # Regressão Polinomial de Grau 4
                    coef = np.polyfit(train_years, train_values, deg=4)
                    poly = np.poly1d(coef)
                    pred = poly(test_years)
                    extrapolated = poly(years_to_predict)
                
                elif method == "spline":
                    # Spline Cúbico
                    cs = CubicSpline(train_years, train_values, extrapolate=True)
                    pred = cs(test_years)
                    extrapolated = cs(years_to_predict)
                
                elif method == "linear_projection":
                    # Projeção Linear baseada na taxa média anual de crescimento
                    growth_rate = (train_values[-1] - train_values[0]) / (train_years[-1] - train_years[0])
                    pred = train_values[-1] + growth_rate * (test_years - train_years[-1])
                    extrapolated = train_values[-1] + growth_rate * (np.array(years_to_predict) - train_years[-1])
                
                # Calcula as métricas de avaliação
                mae = mean_absolute_error(test_values, pred)
                mape = np.mean(np.abs((test_values - pred) / test_values)) * 100
                
                # Armazena as métricas
                metrics['MAE'].append(mae)
                metrics['MAPE'].append(mape)
                
                # Insere os valores extrapolados no DataFrame de resultados
                for year, value in zip(years_to_predict, extrapolated):
                    df_result.loc[state, year] = value
            
            except Exception as e:
                # Em caso de erro, insere NaN nos anos extrapolados e armazena NaN nas métricas
                print(f"Erro ao extrapolar para o estado {state} com método {method}: {e}")
                for year in years_to_predict:
                    df_result.loc[state, year] = np.nan
                metrics['MAE'].append(np.nan)
                metrics['MAPE'].append(np.nan)
        else:
            # Se houver poucos dados, insere NaN nos anos extrapolados e armazena NaN nas métricas
            for year in years_to_predict:
                df_result.loc[state, year] = np.nan
            metrics['MAE'].append(np.nan)
            metrics['MAPE'].append(np.nan)
    
    # Calcula as métricas médias, ignorando NaNs
    avg_metrics = {
        'MAE': np.nanmean(metrics['MAE']),
        'MAPE': np.nanmean(metrics['MAPE'])
    }
    
    return df_result, avg_metrics

# 2. Carregamento e Preparação dos Dados

input_file = os.path.join(data_dir, "combined_population_to_companies.csv")


# Verifica se o arquivo CSV existe no diretório atual
if not os.path.exists(input_file):
    raise FileNotFoundError(f"O arquivo '{input_file}' não foi encontrado no diretório atual.")

# Carrega os dados do CSV
df = pd.read_csv(input_file)

# Filtra os dados para os anos entre 2007 e 2020
df_filtered = df[df['Year'].between(2007, 2020)].copy()

# Verifica se todas as colunas necessárias estão presentes no DataFrame
required_columns = ['SIGLA', 'Year', 'Population', 'Number of Companies']
for col in required_columns:
    if col not in df_filtered.columns:
        raise ValueError(f"A coluna '{col}' está faltando no arquivo CSV.")

# Pivotar os dados separadamente para População e Número de Empresas
df_population = df_filtered.pivot(index='SIGLA', columns='Year', values='Population').astype(float)
df_companies = df_filtered.pivot(index='SIGLA', columns='Year', values='Number of Companies').astype(float)

# Garante que os índices das colunas (anos) sejam inteiros
df_population.columns = df_population.columns.astype(int)
df_companies.columns = df_companies.columns.astype(int)

# 3. Definição dos Métodos de Extrapolação a Serem Utilizados
methods = ['linear_regression', 'polynomial', 'spline', 'linear_projection']

# 4. Inicialização de Dicionários para Armazenar Métricas e Resultados Extrapolados
mae_population = {}
mape_population = {}
extrapolated_population = {}

mae_companies = {}
mape_companies = {}
extrapolated_companies = {}

# 5. Aplicar Extrapolação para População Usando Cada Método
for method in methods:
    # Extrapola a População usando o método atual
    extrapolated_population[method], metrics = extrapolate_series(df_population, method)
    # Armazena as métricas MAE e MAPE
    mae_population[method] = metrics['MAE']
    mape_population[method] = metrics['MAPE']

# 6. Aplicar Extrapolação para Número de Empresas Usando Cada Método
for method in methods:
    # Extrapola o Número de Empresas usando o método atual
    extrapolated_companies[method], metrics = extrapolate_series(df_companies, method)
    # Armazena as métricas MAE e MAPE
    mae_companies[method] = metrics['MAE']
    mape_companies[method] = metrics['MAPE']

# 7. Seleção do Melhor Método com Base nas Métricas (MAE e MAPE)
def select_best_method(metrics_df):
    """
    Seleciona o melhor método com base em múltiplas métricas (MAE e MAPE).
    Escolhe o método com menor MAE e, em caso de empate, menor MAPE.
    
    Parâmetros:
    - metrics_df (pd.DataFrame): DataFrame com as métricas para cada método.
    
    Retorna:
    - str: Nome do melhor método.
    - pd.DataFrame: DataFrame com as métricas para cada método ordenado.
    """
    # Ordena o DataFrame primeiro por MAE e depois por MAPE
    metrics_sorted = metrics_df.sort_values(by=['MAE', 'MAPE'])
    
    # Seleciona o método com a menor MAE e, em caso de empate, a menor MAPE
    best_method = metrics_sorted.iloc[0]['Method']
    
    return best_method, metrics_sorted

# Combina as métricas em DataFrames para População
metrics_population_df = pd.DataFrame({
    'Method': list(methods),
    'MAE': [mae_population[m] for m in methods],
    'MAPE': [mape_population[m] for m in methods]
})

# Combina as métricas em DataFrames para Empresas
metrics_companies_df = pd.DataFrame({
    'Method': list(methods),
    'MAE': [mae_companies[m] for m in methods],
    'MAPE': [mape_companies[m] for m in methods]
})

# Seleciona o melhor método para População
best_population_method, metrics_population_df = select_best_method(metrics_population_df)

# Seleciona o melhor método para Número de Empresas
best_companies_method, metrics_companies_df = select_best_method(metrics_companies_df)

# 8. Imprime as Métricas e o Melhor Método Selecionado
print("\n### MAE e MAPE Médio por Método (População): ###")
print(metrics_population_df[['Method', 'MAE', 'MAPE']])

print(f"\nMelhor Método para População: {best_population_method.replace('_', ' ').capitalize()}")

print("\n### MAE e MAPE Médio por Método (Número de Empresas): ###")
print(metrics_companies_df[['Method', 'MAE', 'MAPE']])

print(f"\nMelhor Método para Número de Empresas: {best_companies_method.replace('_', ' ').capitalize()}")

# 9. Seleciona os Resultados Extrapolados Usando os Melhores Métodos
best_population = extrapolated_population[best_population_method].reset_index()
best_companies = extrapolated_companies[best_companies_method].reset_index()

# 10. Transforma os Dados para o Formato Longo (Tidy) para Facilitar a Combinação
best_population_melted = best_population.melt(id_vars=["SIGLA"], var_name="Year", value_name="Population")
best_companies_melted = best_companies.melt(id_vars=["SIGLA"], var_name="Year", value_name="Number of Companies")

# 11. Combina os Dados Extrapolados de População e Número de Empresas
df_final = pd.merge(best_population_melted, best_companies_melted, on=["SIGLA", "Year"], how="left")

# 12. Calcula a Razão População/Empresas
df_final["Population-to-Companies Ratio"] = df_final["Population"] / df_final["Number of Companies"]

# 13. Correção de Valores Infinitos e Tipos de Dados
# Identifica as colunas numéricas para aplicar as correções
numeric_cols = df_final.select_dtypes(include=[np.number]).columns.tolist()

# Substitui valores infinitos por NaN apenas nas colunas numéricas
df_final[numeric_cols] = df_final[numeric_cols].replace([np.inf, -np.inf], np.nan)

# Garante que as colunas numéricas são do tipo float antes da interpolação
df_final[numeric_cols] = df_final[numeric_cols].astype(float)

# Interpola dados faltantes utilizando interpolação linear apenas nas colunas numéricas
df_final[numeric_cols] = df_final[numeric_cols].interpolate(method='linear', inplace=False)

# 14. Salva os Resultados Extrapolados em um Arquivo CSV
output_file = "extrapolated_population_companies_separate_methods.csv"
df_final.to_csv(output_file, index=False)
print(f"\nExtrapolação salva em '{output_file}'")

# 15. Geração de Gráficos da Razão População/Empresas ao Longo dos Anos
output_dir = "charts_ratio"
os.makedirs(output_dir, exist_ok=True)  # Cria a pasta se não existir

# Itera sobre cada estado para gerar e salvar os gráficos
for state in df_final["SIGLA"].unique():
    df_state = df_final[df_final["SIGLA"] == state].sort_values("Year")
    
    # Separa os dados históricos e extrapolados
    historical = df_state[df_state["Year"] <= 2020]
    extrapolated = df_state[df_state["Year"] > 2020]
    
    plt.figure(figsize=(6, 3))
    
    # Plotar dados históricos em azul com linha sólida
    plt.plot(historical["Year"], historical["Population-to-Companies Ratio"], 'o-b', label="Histórico")
    
    # Verifica se há dados extrapolados para plotar
    if not extrapolated.empty and not historical.empty:
        # Obtém o último ponto histórico (2020)
        last_year_historical = historical["Year"].max()
        last_value_historical = historical[historical["Year"] == last_year_historical]["Population-to-Companies Ratio"].values[0]
        
        # Obtém o primeiro ponto extrapolado (2021)
        first_year_extrapolated = extrapolated["Year"].min()
        first_value_extrapolated = extrapolated[extrapolated["Year"] == first_year_extrapolated]["Population-to-Companies Ratio"].values[0]
        
        # Conecta 2020 a 2021 com linha tracejada vermelha
        plt.plot([last_year_historical, first_year_extrapolated],
                 [last_value_historical, first_value_extrapolated],
                 'r--')
    
    # Plotar dados extrapolados em vermelho com linha tracejada
    plt.plot(extrapolated["Year"], extrapolated["Population-to-Companies Ratio"], 'o--r', label="Extrapolado")
    
    # Configurações do gráfico
    plt.title(f"Evolução da Razão População/Empresas - {state}")
    plt.xlabel("Ano")
    plt.ylabel("Razão População/Empresas")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    
    # Salva o gráfico na pasta especificada
    plt.savefig(f"{output_dir}/{state}_ratio.png")
    plt.show()

print(f"Gráficos da Razão População/Empresas salvos na pasta '{output_dir}'")


# **Comparação dos Métodos de Extrapolação**

## **Resultados da Extrapolação**

### **População**
| Método               | MAE (Janela 5 anos) | MAPE (Janela 5 anos) | MAE (Histórico Completo) | MAPE (Histórico Completo) |
|----------------------|--------------------|--------------------|----------------------|----------------------|
| **Spline**          | **924.93**  | **0.195%** | 63225.57 | 3.752% |
| **Polynomial**      | 5176.17 | 0.254% | **60831.10** | **3.662%** |
| Linear Projection   | 5005.15 | 0.420% | 75106.92 | 4.627% |
| Linear Regression   | 10276.74 | 0.881% | 76141.25 | 4.678% |

- **Melhor Método (Janela de 5 anos):** **Spline**
- **Melhor Método (Histórico Completo):** **Polynomial**

---

### **Número de Empresas**
| Método              | MAE (Janela 5 anos) | MAPE (Janela 5 anos) | MAE (Histórico Completo) | MAPE (Histórico Completo) |
|---------------------|--------------------|--------------------|----------------------|----------------------|
| **Linear Projection**  | **148.08**  | **19.97%** | **113.71** | **10.90%** |
| Polynomial         | 446.97 | 29.17% | 3587.56 | 425.69% |
| Linear Regression  | 482.12 | 42.29% | 115.08  | 13.27% |
| Spline            | 1354.58 | 106.61% | 1844.86 | 238.47% |

- **Melhor Método (Janela de 5 anos):** **Linear Projection**
- **Melhor Método (Histórico Completo):** **Linear Projection**


### Resultado: A abordagem do histórico completo obteve melhores resultados que a janela de 5 anos.

---
# **Clusterização e Identificação de Oportunidades**

## **Objetivo**
Agrupar os estados brasileiros em **três categorias** com base na razão **População/Empresas**, utilizando o algoritmo **K-Means**. 

1. **Oportunidade** (Alta razão População/Empresas) → Muitas pessoas por empresa → Mercado pouco explorado e grande potencial de crescimento
2. **Estável** (Média razão População/Empresas) → Equilíbrio entre pessoas por empresa
3. **Saturado** (Baixa razão População/Empresas) → Muitas empresas por pessoas → Mercado altamente competitivo e pouco espaço para novas empresas

---

## **Análise de Dados (2007 - 2022)**
- Utilizei dados de **2007 a 2022**, sendo os dados de 2021 e 2022 provenientes de extrapolação.
- O número ideal de clusters foi determinado a partir do método do cotovelo. Com valores de k entre 1 e 10.

---

## **Clusterização dos Estados**
Após definir o número de clusters, o K-Means foi aplicado para segmentar os estados e classificar os grupos:

| Cluster | Categoria        | Característica |
|---------|-----------------|---------------|
| 2       | Oportunidade    | **Poucas empresas para muitas pessoas** → Maior demanda do que oferta, ideal para expansão. |
| 1       | Estável         | **Mercado equilibrado** entre empresas e população. |
| 0       | Saturado        | **Muitas empresas para poucas pessoas** → Alta concorrência e pouca margem para novos negócios. |

### **Distribuição dos Estados**
Criação de um gráfico de barras para visualizar quantos estados pertencem a cada cluster.

---

## **Visualização dos Clusters**
### **A. Gráfico de Dispersão**
Um gráfico para visualizar os **clusters** e sua relação com a razão **População/Empresas** por ano. 


### **B. Mapa do Brasil com Clusters**
Gerei um **mapa interativo** do Brasil, onde os estados foram coloridos conforme a categoria do cluster.

- **Estados com Maior Oportunidade**: Azul 
- **Estados Estáveis**: Verde 
- **Estados Saturados**: Vermelho 


## **C. Correlação entre População e Empresas**
É plotado em um gráfico de barras a correlação entre população e número de empresas.
ation_by_state.csv** → Contém a correlação entre população e empresas.

---


In [ ]:

# 1. Carregar os dados
df = df_final

# 2. Verificar as colunas disponíveis
print("Colunas encontradas no dataset:", df.columns.tolist())

# 3. Garantir que os anos sejam reconhecidos corretamente
df["Year"] = df["Year"].astype(int)

# 4. Selecionar as colunas relevantes para clusterização
features = ["SIGLA", "Year", "Population", "Number of Companies", "Population-to-Companies Ratio"]
df = df[features].dropna()  # Remover linhas com valores ausentes

# 5. Normalizar os dados de entrada 
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[["Population-to-Companies Ratio"]])

# 6. Determinar o número ideal de clusters pelo método do cotovelo
inertia = []
K_range = range(1, 10)  # Testar de 1 a 10 clusters

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_scaled)
    inertia.append(kmeans.inertia_)

# 7. Plotar o método do cotovelo
plt.figure(figsize=(6, 4))
plt.plot(K_range, inertia, marker='o', linestyle='--')
plt.xlabel("Número de Clusters (k)")
plt.ylabel("Inércia")
plt.title("Método do Cotovelo para Determinação de k")
plt.grid()
plt.show()

# 8. Aplicar K-Means com o número escolhido de clusters
optimal_k = int(input("Digite o número de clusters ideal baseado no gráfico do cotovelo: "))
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df["Cluster"] = kmeans.fit_predict(df_scaled)

# 9. Criar um DataFrame final contendo os pontos com suas classificações
df_clusters = df[["SIGLA", "Year", "Population", "Number of Companies", "Population-to-Companies Ratio", "Cluster"]]

# A. Gráfico de Dispersão

# 10. Visualizar os clusters em um gráfico de dispersão
plt.figure(figsize=(8, 5))
sns.scatterplot(
    data=df_clusters,
    x="Year",
    y="Population-to-Companies Ratio",
    hue="Cluster",
    palette="tab10"
)
plt.title("Clusterização dos Estados com Base na Razão População/Empresas")
plt.xlabel("Ano")
plt.ylabel("Razão População/Empresas")
plt.legend(title="Cluster")
plt.grid(alpha=0.5, linestyle="--")
plt.tight_layout()
plt.show()

# 11. Salvar os resultados em CSV
file_path = os.path.join(data_dir, "state_clusters.csv")
df_clusters.to_csv(file_path, index=False)
print("\nClusterização concluída! Resultados salvos em 'state_clusters_full.csv'.")


### Mapa do Brasil com Clusters

In [ ]:
# B. Mapa do Brasil com Clusters

import os
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as mcolors

# 1. Definir o diretório onde está o shapefile extraído
shapefile_dir = "shapefile_brasil"

# 2. Localizar o arquivo .shp dentro da pasta extraída
shapefile_path = None
for file in os.listdir(shapefile_dir):
    if file.endswith(".shp"):
        shapefile_path = os.path.join(shapefile_dir, file)
        break

if shapefile_path is None:
    raise FileNotFoundError("Nenhum arquivo .shp encontrado na pasta extraída!")

# 3. Carregar o shapefile dos estados do Brasil
brasil_map = gpd.read_file(shapefile_path)

# 4. Filtrar apenas os estados brasileiros
brasil_map = brasil_map[brasil_map['admin'] == "Brazil"]

# 5. Tabela de clusters já carregada em memoria

# 6. Garantir que a coluna 'SIGLA' está presente no dataset
if "SIGLA" not in df_clusters.columns:
    raise ValueError("A coluna 'SIGLA' não está presente no dataset 'state_clusters.csv'.")

# 7. Renomear a coluna SIGLA para postal para compatibilidade com o shapefile
df_clusters.rename(columns={"SIGLA": "postal"}, inplace=True)

# 8. Verificar se a coluna "Cluster" existe no dataset
if "Cluster" not in df_clusters.columns:
    raise ValueError("A coluna 'Cluster' não está presente no dataset.")

# 9. Criar um dicionário de categorias baseado nos clusters
cluster_mapping = {
    2: "Oportunidade",   # Poucas empresas para muitas pessoas → Maior demanda, ideal para expansão.
    1: "Estável",        # Equilíbrio entre empresas e população.
    0: "Saturado"        # Muitas empresas para poucas pessoas → Alta concorrência.
}

# 10. Criar uma nova coluna "Category" com os nomes das classificações
df_clusters["Category"] = df_clusters["Cluster"].map(cluster_mapping)

# 11. Unir o shapefile com os clusters
brasil_map = brasil_map.merge(df_clusters, on="postal", how="left")

# 12. Definir um colormap personalizado para os clusters
cluster_colors = {
    "Oportunidade": "blue",
    "Estável": "green",
    "Saturado": "red"
}

# 13. Criar um mapa colorido por cluster
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
brasil_map.plot(column="Category", cmap=mcolors.ListedColormap(cluster_colors.values()), legend=True, ax=ax, edgecolor="black")

# 14. Adicionar as siglas dos estados no centro geométrico de cada estado
for idx, row in brasil_map.iterrows():
    plt.text(
        row.geometry.centroid.x, 
        row.geometry.centroid.y, 
        row["postal"],  # Sigla do estado
        fontsize=8, fontweight="bold", ha="center", va="center", color="black"
    )

# 15. Ajustar a legenda manualmente
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label)
           for label, color in cluster_colors.items()]
ax.legend(handles=handles, title="Clusters")

plt.title("Classificação de Clusters dos Estados do Brasil")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()







In [ ]:


# 1. Carregar o dataset com valores extrapolados
combined_df = df_final

# 2. Garantir que a coluna 'Year' esteja presente e no formato correto
if "Year" not in combined_df.columns:
    raise ValueError("A coluna 'Year' não está presente no dataset.")

combined_df["Year"] = pd.to_numeric(combined_df["Year"], errors="coerce")

# 3. Garantir que 'Population-to-Companies Ratio' está no dataset
if "Population-to-Companies Ratio" not in combined_df.columns:
    raise ValueError("A coluna 'Population-to-Companies Ratio' não está presente no dataset.")

# 4. Gráfico Interativo de Linha: Evolução da Razão População/Empresas por Estado ao Longo dos Anos
fig_line = px.line(
    combined_df,
    x="Year",
    y="Population-to-Companies Ratio",
    color="SIGLA",  # Diferente cor para cada estado
    title="Evolução da Razão População/Empresas por Estado (2007-2022)",
    labels={"Population-to-Companies Ratio": "Razão População/Empresas", "Year": "Ano"},
    hover_name="SIGLA"
)
fig_line.update_layout(hovermode="x unified")
fig_line.show()

# 5. Gráfico Interativo de Barras: Razão Média por Estado (2007-2022)
mean_ratio_by_state = combined_df.groupby("SIGLA")["Population-to-Companies Ratio"].mean().reset_index()

fig_bar = px.bar(
    mean_ratio_by_state,
    x="SIGLA",
    y="Population-to-Companies Ratio",
    title="Razão Média População/Empresas por Estado (2007-2022)",
    labels={"Population-to-Companies Ratio": "Razão Média", "SIGLA": "Estado"},
    color="SIGLA",
    hover_name="SIGLA"
)
fig_bar.update_layout(showlegend=False)
fig_bar.show()

# 6. Gráfico Interativo de Dispersão: Correlação entre População e Número de Empresas
# Carregar a base original para população e empresas
input_file = os.path.join(data_dir, "combined_population_to_companies.csv")
original_df = pd.read_csv(input_file)

# Verificar se as colunas essenciais estão presentes
required_columns = ["SIGLA", "Year", "Population", "Number of Companies"]
for col in required_columns:
    if col not in original_df.columns:
        raise ValueError(f"A coluna '{col}' não está presente no dataset original.")

# Garantir que 'Year' esteja em formato numérico
original_df["Year"] = pd.to_numeric(original_df["Year"], errors="coerce")

# Selecionar apenas os anos extrapolados (2021 e 2022)
extrapolated_df = combined_df[combined_df["Year"].isin([2021, 2022])]

# Unir dados originais com extrapolados
final_df = pd.concat([original_df, extrapolated_df])

# Criar gráfico de dispersão
fig_scatter = px.scatter(
    final_df,
    x="Number of Companies",
    y="Population",
    color="SIGLA",
    title="Correlação entre População e Número de Empresas (Incluindo 2021-2022)",
    labels={"Number of Companies": "Número de Empresas", "Population": "População"},
    hover_name="SIGLA"
)
fig_scatter.show()



In [ ]:
# C. Correlação entre População e Empresas

# 1. Dados já carregados em memoria
# combined_df

# 2. Carregar os dados originais para população e empresas
file_path = os.path.join(data_dir, "combined_population_to_companies.csv")
original_df = pd.read_csv(file_path)

# 3. Filtrar apenas os dados de 2021 e 2022
extrapolated_df = combined_df[combined_df["Year"].isin([2021, 2022])]

# 4. Unir dados originais com extrapolados
final_df = pd.concat([original_df, extrapolated_df], ignore_index=True)

# 5. Calcular a correlação entre População e Número de Empresas para cada estado
correlation_results = []
for state, group in final_df.groupby("SIGLA"):
    if "Population" in group.columns and "Number of Companies" in group.columns:
        if group["Population"].nunique() > 1 and group["Number of Companies"].nunique() > 1:
            correlation = group["Population"].corr(group["Number of Companies"])
        else:
            correlation = None  # Se há apenas um ponto, a correlação não pode ser calculada
        correlation_results.append({"State": state, "Correlation": correlation})

# 6. Criar DataFrame com os resultados
correlation_by_state = pd.DataFrame(correlation_results)
correlation_by_state.dropna(inplace=True)  # Remover valores nulos

# 7. Salvar os resultados
file_path = os.path.join(data_dir, "correlation_by_state.csv")
correlation_by_state.to_csv(file_path, index=False)

# 8. Criar um gráfico interativo de correlação
fig_correlation = px.bar(
    correlation_by_state,
    x="State",
    y="Correlation",
    title="Correlação entre População e Número de Empresas por Estado (Incluindo 2021-2022)",
    labels={"State": "Estado", "Correlation": "Correlação"},
    color="Correlation",
    color_continuous_scale=["red", "green", "blue"],  # Vermelho negativo, verde neutro, azul positivo
)

# Exibir o gráfico interativo
fig_correlation.show()





## **Resultados e relatório**
[Relatório](document.pdf)
